In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Create a dataframe to be used as the base of a power prediction model
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
start_time = pd.to_datetime('2014-05-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
#infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_359350"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [19]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

# clean user names
jobs['user'] = jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

jobs_to_nodes_whole_data contains 469095 records
There are 336787 long jobs in the Alina's interval


In [20]:
jobs.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000,1,16,14,...,NaN,0.0,True,1.0,0,1,0,631,631,0


In [21]:
jobs_q1 = jobs[jobs['real_pow'] > 0]

In [26]:

#target = jobs['real_pow']

# drop unused (or not usable) info
data = jobs_q1.drop(['job_id_string'], axis=1)
data = data.drop(['start_time'], axis=1)
data = data.drop(['job_tot_timepoints'], axis=1)
data = data.drop(['job_timepoints'], axis=1)
data = data.drop(['good_nodes'], axis=1) 
data = data.drop(['exit_status'], axis=1)
data = data.drop(['pbs_exit_code'], axis=1)
data = data.drop(['deleted'], axis=1)
data = data.drop(['dependency'], axis=1)
data = data.drop(['real_pow_quality'], axis=1)
data = data.drop(['time_req'], axis=1)
#data = data.drop(['real_pow'], axis=1)
#data = data.drop(['job_name'], axis=1)

data = data.drop(['gpu_req'], axis=1)
data = data.drop(['mic_req'], axis=1)

data.keys()

Index(['job_name', 'queue', 'run_start_time', 'end_time', 'user', 'node_req',
       'cpu_req', 'mem_req', 'real_pow', 'ran_alone', 'n_2_1', 'n_2_2',
       'n_3_1'],
      dtype='object')

In [54]:
jobs_train = data[pd.to_datetime(data['end_time']) <= train_end_time]
jobs_train = jobs_train[pd.to_datetime(jobs_train['run_start_time']) > train_start_time]
print("train set contains {} records".format(jobs_train.shape[0]))

train_target = jobs_train['real_pow']
jobs_train = jobs_train.drop(['run_start_time'], axis=1)
jobs_train = jobs_train.drop(['end_time'], axis=1)
jobs_train = jobs_train.drop(['real_pow'], axis=1)

jobs_test = data[pd.to_datetime(data['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) > train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))
test_target = jobs_test['real_pow']
jobs_test = jobs_test.drop(['run_start_time'], axis=1)
jobs_test = jobs_test.drop(['end_time'], axis=1)
jobs_test = jobs_test.drop(['real_pow'], axis=1)

train set contains 286253 records
test set contains 24918 records


In [55]:
data.head(5)

,job_name,queue,run_start_time,end_time,user,node_req,cpu_req,mem_req,real_pow,ran_alone,n_2_1,n_2_2,n_3_1
job_id,,,,,,,,,,,,,
2,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo,1,16,14,213.907235,True,1,0,0
3,7ribb-relax,parallel,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0,6,96,84,1232.722332,True,5,1,0
4,rth2e6t0DC,parallel,2014-03-31 01:00:02,2014-03-31 05:01:18,alupi000,1,16,14,204.154343,True,1,0,0
8,INIT,R472836,2014-03-31 04:00:17,2014-03-31 07:50:11,a07smr01,1,1,2,4.619396,False,0,0,1
9,LM_PRE,R472836,2014-03-31 04:00:18,2014-03-31 04:23:30,a07smr01,11,176,154,3437.597435,True,0,0,11


In [56]:
print(data.shape[0])

311172


In [57]:
#data['time_req'].unique()

In [43]:
#data[data['time_req'].isnull()].shape[0]

In [44]:
#data.loc[data['time_req'] == '--', 'time_req'] = np.nan

In [45]:
#y = data.time_req.str.split(':')   
#i, j = y.str[0], y.str[1]  
#
#data['time_req'] = i.astype(int) + (j.astype(float) / 60) 
#data.head(5)


In [53]:
#debug
jobs_train['job_name'].unique().shape[0]

8890

In [58]:
from sklearn.feature_extraction import FeatureHasher
import copy
from sklearn.ensemble import RandomForestRegressor


X_train_hash = copy.copy(jobs_train)
X_test_hash = copy.copy(jobs_test)
for i in range(X_train_hash.shape[1]):
    X_train_hash.iloc[:,i]=X_train_hash.iloc[:,i].astype('str')
for i in range(X_test_hash.shape[1]):
    X_test_hash.iloc[:,i]=X_test_hash.iloc[:,i].astype('str')
h = FeatureHasher(n_features=1000, input_type="string")
X_train_hash = h.transform(X_train_hash.values)
X_test_hash = h.transform(X_test_hash.values)



/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
regr = RandomForestRegressor()
regr.fit(X_train_hash,train_target)


#l.fit(X_train_hash,y_train)
#y_pred = l.predict_proba(X_test_hash)
#print(log_loss(y_test,y_pred))#0.4

In [60]:
from sklearn.model_selection import cross_val_score
print("[RandomForestRegressor] Train data Accuracy: %0.2f " % regr.score(X_train_hash, train_target))


print("[RandomForestRegressor] Test data Accuracy: %0.2f " % regr.score(X_test_hash, test_target))

[RandomForestRegressor] Train data Accuracy: 0.81 
[RandomForestRegressor] Test data Accuracy: 0.68 


In [64]:
test_predictions = regr.predict(X_test_hash)
test_predictions.shape[0]

24918

In [65]:
jobs_test_full = data[pd.to_datetime(data['end_time']) <= test_end_time]
jobs_test_full = jobs_test_full[pd.to_datetime(jobs_test_full['run_start_time']) > train_end_time]

mean_real_pow = jobs_test_full['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))

jobs_test_full['numerator'] = (jobs_test_full['real_pow'] - test_predictions)**2
jobs_test_full['denumerator'] = (jobs_test_full['real_pow'] - mean_real_pow)**2
N = jobs_test_full['numerator'].shape[0]

r_2 = 1 - (jobs_test_full['numerator'].sum() / jobs_test_full['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(jobs_test_full['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 74.0599653418435
r_2 is 0.6761439322256165
RMSE is 91.53533864787914
NRMSE is 1.2359624829065665


In [61]:
# print(X_train_hash)

In [46]:
#qs = []
#for q in data['queue']:
#    if not q in qs:
#        qs.append(q)
#us = []
#for u in data['user']:
#    if not u in us:
#        us.append(u)
#jns = []
#for jn in data['job_name']:
#    if not jn in jns:
#        jns.append(jn)

# convert categorical data
#print('job_name')
#dms = pd.get_dummies(data['job_name'])
#data = pd.concat([data, dms], axis=1)    
#data = data.drop(['job_name'], axis=1)

print('queue')
dms = pd.get_dummies(data['queue'])
data = pd.concat([data, dms], axis=1)    
data = data.drop(['queue'], axis=1)

print('user')
dms = pd.get_dummies(data['user'])
data = pd.concat([data, dms], axis=1)    
data = data.drop(['user'], axis=1)


# delete rows  with missing values
data = data.dropna()
#target = target.dropna()

target = data['real_pow']
data = data.drop(['real_pow'], axis=1)

features_list = list(data.columns)
# print(features_list)

# convert panda dataframe into numpy arrays
data_np = np.array(data)
target_np = np.array(target)

# scale (obtain data with zero mean and unit variance)
#data_scaled = preprocessing.scale(data_np)
#target_scaled = preprocessing.scale(target_np)
data_scaled = data_np
target_scaled = target_np

data.iloc[:,5:].head(5)

queue
user


,n_2_2,n_3_1,R1010870,R1010998,R1011000,R1013900,R1013901,R1013902,R1013903,R1013904,...,tbusatta,tgiorgin,tjones00,tlucchin,vcalzia0,vdalcol0,vmarino0,vmiglior,vruggie1,wxie0000
job_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
print(data_scaled.shape[0])
print(target_scaled.shape[0])

274889
274889


In [32]:
from sklearn.model_selection import train_test_split

# create test and training set
train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled, test_size = 0.2, random_state = 42)

In [33]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(train_data,train_target)



/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
from sklearn.model_selection import cross_val_score
print("[RandomForestRegressor] Train data Accuracy: %0.2f " % regr.score(train_data, train_target))


print("[RandomForestRegressor] Test data Accuracy: %0.2f " % regr.score(test_data, test_target))

[RandomForestRegressor] Train data Accuracy: 0.74 
[RandomForestRegressor] Test data Accuracy: 0.70 
